# Collect NBA schedule information

In [81]:
import os
import pandas 
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

## Collect team information

In [82]:
%run 1-teams.ipynb

In [83]:
dic_team.keys()

dict_keys(['team', 'home', 'stats', 'schedule', 'roster', 'depth', 'abv'])

In [84]:
dt.head()
DT = dt.copy()

### Set url schema
* home: http://www.espn.com/nba/team/_/name/bos/boston-celtics
* schedule: http://www.espn.com/nba/team/schedule/_/name/bos/boston-celtics
* /nba/teams/schedule?team=bos
* add espn.com to status url

In [89]:
DT['schedule'] = DT['home'].replace({'_': 'schedule/_'}, regex=True)
DT.head()

,team,home,stats,schedule,roster,depth
bos,Boston Celtics,http://www.espn.com/nba/team/_/name/bos/boston...,/nba/teams/stats?team=bos,http://www.espn.com/nba/team/schedule/_/name/b...,/nba/teams/roster?team=bos,/nba/teams/depth?team=bos
bkn,Brooklyn Nets,http://www.espn.com/nba/team/_/name/bkn/brookl...,/nba/teams/stats?team=bkn,http://www.espn.com/nba/team/schedule/_/name/b...,/nba/teams/roster?team=bkn,/nba/teams/depth?team=bkn
nyk,New York Knicks,http://www.espn.com/nba/team/_/name/ny/new-yor...,/nba/teams/stats?team=nyk,http://www.espn.com/nba/team/schedule/_/name/n...,/nba/teams/roster?team=nyk,/nba/teams/depth?team=nyk
phi,Philadelphia 76ers,http://www.espn.com/nba/team/_/name/phi/philad...,/nba/teams/stats?team=phi,http://www.espn.com/nba/team/schedule/_/name/p...,/nba/teams/roster?team=phi,/nba/teams/depth?team=phi
tor,Toronto Raptors,http://www.espn.com/nba/team/_/name/tor/toront...,/nba/teams/stats?team=tor,http://www.espn.com/nba/team/schedule/_/name/t...,/nba/teams/roster?team=tor,/nba/teams/depth?team=tor


### Data collection
1. Set global variables
* Set data architecture
* Loop through teams
* Append team url information to schedule url
* Collect schedule information
* Store information in dictionary
* Convert dictionary to dataframe
* Convert dataframe to csv
* Save csv to file

In [96]:
URL = DT.schedule[0]
# http://www.espn.com/nba/team/schedule/_/name/bos/boston-celtics    
r = requests.get(URL)
soup = BeautifulSoup(r.text, 'lxml')
soup.table

<table cellpadding="3" cellspacing="1" class="tablehead"><tr class="stathead"><td colspan="9">2018 Regular Season Schedule</td></tr><tr class="colhead"><td>OCTOBER</td><td>OPPONENT</td><td>RESULT</td><td>W-L</td><td align="left" title="Points Leader">HI POINTS</td><td align="left" title="Rebounds Leader">HI REBOUNDS</td><td align="left" title="Assists Leader">HI ASSISTS</td></tr><tr class="oddrow team-46-5"><td>Tue, Oct 17</td><td><ul class="game-schedule"><li class="game-status">@</li><li class="team-logo-small logo-nba-small"><a href="http://www.espn.com/nba/team/_/name/cle/cleveland-cavaliers"><img src="http://a.espncdn.com/combiner/i?img=/i/teamlogos/nba/500/scoreboard/Cle.png&amp;h=80&amp;w=80"/></a></li><li class="team-name"><a href="http://www.espn.com/nba/team/_/name/cle/cleveland-cavaliers">Cleveland</a></li></ul></td><td><ul class="game-schedule"><li class="game-status loss"><span class="redfont">L</span></li><li class="score"><a href="//www.espn.com/nba/recap/_/id/400974437"

In [91]:
BASE_URL = dt.schedule

match_id = []
dates = []
home_team = []
home_team_score = []
visit_team = []
visit_team_score = []

for URL in BASE_URL[0:4]:
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    print(soup)

MissingSchema: Invalid URL '/nba/teams/schedule?team=bos': No schema supplied. Perhaps you meant http:///nba/teams/schedule?team=bos?

In [78]:
BASE_URL = dt.schedule

match_id = []
dates = []
home_team = []
home_team_score = []
visit_team = []
visit_team_score = []

for URL in BASE_URL[0:4]:
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    table = soup.table

    for row in table.find_all('tr')[1:]: # Remove header
        columns = row.find_all('td')
        try:
            _home = True if columns[1].li.text == 'vs' else False
            _other_team = columns[1].find_all('a')[1].text
            _score = columns[2].a.text.split(' ')[0].split('-')
            _won = True if columns[2].span.text == 'W' else False

            match_id.append(columns[2].a['href'].split('?id=')[1])
            home_team.append(_team if _home else _other_team)
            visit_team.append(_team if not _home else _other_team)
            d = datetime.strptime(columns[0].text, '%a, %b %d')
            dates.append(date(year, d.month, d.day))

            if _home:
                if _won:
                    home_team_score.append(_score[0])
                    visit_team_score.append(_score[1])
                else:
                    home_team_score.append(_score[1])
                    visit_team_score.append(_score[0])
            else:
                if _won:
                    home_team_score.append(_score[1])
                    visit_team_score.append(_score[0])
                else:
                    home_team_score.append(_score[0])
                    visit_team_score.append(_score[1])
        except Exception as e:
            pass # Not all columns row are a match, is OK
            # print(e)

dic = {'id': match_id, 'date': dates, 'home_team': home_team, 'visit_team': visit_team,
        'home_team_score': home_team_score, 'visit_team_score': visit_team_score}
dic.keys()

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
match_id = []
dates = []
home_team = []
home_team_score = []
visit_team = []
visit_team_score = []

for URL in BASE_URL[0:4]:
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, 'lxml')
    table = soup.table

    for row in table.find_all('tr')[1:]: # Remove header
        columns = row.find_all('td')
        try:
            _home = True if columns[1].li.text == 'vs' else False
            _other_team = columns[1].find_all('a')[1].text
            _score = columns[2].a.text.split(' ')[0].split('-')
            _won = True if columns[2].span.text == 'W' else False

            match_id.append(columns[2].a['href'].split('?id=')[1])
            home_team.append(_team if _home else _other_team)
            visit_team.append(_team if not _home else _other_team)
            d = datetime.strptime(columns[0].text, '%a, %b %d')
            dates.append(date(year, d.month, d.day))

            if _home:
                if _won:
                    home_team_score.append(_score[0])
                    visit_team_score.append(_score[1])
                else:
                    home_team_score.append(_score[1])
                    visit_team_score.append(_score[0])
            else:
                if _won:
                    home_team_score.append(_score[1])
                    visit_team_score.append(_score[0])
                else:
                    home_team_score.append(_score[0])
                    visit_team_score.append(_score[1])
        except Exception as e:
            pass # Not all columns row are a match, is OK
            # print(e)

dic = {'id': match_id, 'date': dates, 'home_team': home_team, 'visit_team': visit_team,
        'home_team_score': home_team_score, 'visit_team_score': visit_team_score}
dic.keys()

In [13]:
games = pandas.DataFrame(dic)
print(games)

Empty DataFrame
Columns: [date, home_team, home_team_score, id, visit_team, visit_team_score]
Index: []
